![](https://ai-studio-static-online.cdn.bcebos.com/c5d5b0e941ab46a783d853d2724f9986442cc985dda74222a64053349cd6778b)

## Abstract（摘要）
哈咯，大家好，我是生而为弟。本期我们主要介绍一下苏大的R-Drop。

众所周知，在训练深度神经网络时，正则化技术对于防止过拟合和提高深度模型的泛化能力是必不可少的。其中，dropout 技术是使用最广泛的一种，旨在通过在训练期间简单地从神经网络中删除一定比例的隐藏单元来避免协同适应并执行隐式集成。

好，现在我们已知dropout的牛逼，然而，在实作中，dropout往往是难以找到最优解。试想每一次前向计算，dropout都会随机丢弃一些隐藏单元，所有的丢弃的可能情况N可以用排列组合来计算，也正因如此，模型的参数空间也扩大了N倍，因此要找到最优解需要更多轮的训练以及更精细的调参。因此，苏大的大佬们一拍脑袋，想了个法子，既能充分利用dropout来应对过拟合，也能尽可能地缩小参数空间。

## Introduction（介绍）
首先有一个先验知识，大佬们认为，每一次前向计算都会随机丢掉一些隐藏单元，因此其实在每一轮计算时，模型的结构都是不同的。好，知道了这一点，下面我们继续。

平时我们在前向计算中都是只dropout一次，这回大佬们开了个脑洞，我们dropout两次试试。

具体来说，在每次 mini-batch 训练中，每个数据样本都经过两次前向传递，每次传递都由不同的子模型通过随机丢弃一些隐藏单元来处理。 R-Drop 通过最小化两个分布之间的双向KL散度，强制两个子模型输出的相同数据样本的两个分布彼此一致。也就是说，R-Drop正则化了模型训练中每个数据样本的两个采样子模型的输出。

下图形象地展示了该思想
![](https://ai-studio-static-online.cdn.bcebos.com/ee4a793312d64b99826a26c5fc178f05f62a399db5c748229fe693d551a34401)

## Algorithm（算法）
![](https://ai-studio-static-online.cdn.bcebos.com/312b7eabfb204582aa5a4f2517401ddee051d856cfd542a1bd4e111f6b3268c4)
![](https://ai-studio-static-online.cdn.bcebos.com/7fb704dc37444d4b98d6da5809e3ad7bef439937354b491eb9e1aede614ffa8d)
![](https://ai-studio-static-online.cdn.bcebos.com/30c4912aef764beab6a73a27de6d12a1d05f073505fc4d83b84d4302a1741264)
![](https://ai-studio-static-online.cdn.bcebos.com/0e7481c71feb4d7e8a840f0b3db2b481c13ca8446d35481b8e891041ef808e5b)


## Implement（实作）

### 解压数据集

In [ ]:
!unzip -oq /home/aistudio/data/data70856/cat_12_test.zip -d data/
!unzip -oq /home/aistudio/data/data70856/cat_12_train.zip -d data/
!mv data/work/cat_12_train data/
!mv data/work/cat_12_test data/
!rm -rf data/work/

### 导入所需库

In [ ]:
import numpy as np
import os
import cv2
import paddle
from paddle.io import Dataset,DataLoader
import paddle.nn as nn
import paddle.nn.functional as F
import paddle.vision.transforms as T
import matplotlib.pyplot as plt
import matplotlib.image as Image
from utils import show_imgs

In [ ]:
# txt文件路径，提取出所有图片的路径以及类别
files_path="data/data70857/train_list.txt"
# 测试集文件夹路径
dir_path="data/cat_12_test"
# 批大小
batch_size=32
#学习率
learning_rate=3e-5
# 训练轮数
num_epoch=30
# KL_loss权重
alpha=5

### 数据准备

In [ ]:
train_paths=[]
train_labels=[]
eval_paths=[]
eval_labels=[]
with open(files_path) as f:
    lines=f.readlines()
    flag=0
    # line:每条路径及分类
    for line in lines:
        '''
        此处使用matplotlib读取图片，因为一方面opencv无法识别中文路径
        另一方面，opencv有时无法读取某些图片，原因未知
        '''
        img_path="data/"+line.split("\t")[0]
        img=Image.imread(img_path)
        # 判断图片是否为空
        if not(img is None):
            label=line.split("\t")[1]
            if flag%10==0:
                eval_paths.append(img_path)
                eval_labels.append(int(label))
            else:
                train_paths.append(img_path)
                train_labels.append(int(label))
        flag+=1
train_paths=np.array(train_paths)
print(train_paths.shape)
train_labels=paddle.to_tensor(train_labels,dtype="int64")
print(train_labels.shape)
eval_paths=np.array(eval_paths)
print(eval_paths.shape)
eval_labels=paddle.to_tensor(eval_labels,dtype="int64")
print(eval_labels.shape)

### 数据预处理

In [ ]:
'''
此处使用127.5进行归一化，将图片RGB值放缩至-1～1。
相较于直接除以255，放缩至0～1
该方法使网络的拟合范围增大，可提升模型效果
'''
train_transforms=T.Compose([T.Resize((250,250)),
                        T.ColorJitter(0.2,0.2,0.2,0.2),
                        T.CenterCrop(224),
                        T.RandomHorizontalFlip(0.3),
                        T.RandomRotation(15),
                        T.Transpose(),
                        T.Normalize(mean=127.5,std =127.5)])
eval_transforms=T.Compose([T.Resize((224,224)),
                        T.Transpose(),
                        T.Normalize(mean=127.5,std =127.5)])
img=Image.imread(train_paths[1])
img=train_transforms(img)
print(img.shape)

### 构造数据读取器

In [ ]:
class Mydataset(Dataset):
    def __init__(self,paths,labels,transforms):
        super(Mydataset, self).__init__()
        self.paths=paths
        self.labels=labels
        self.transforms=transforms

    def __getitem__(self,index):
        '''
        因为本项目使用matplotlib读取图片
        因此图片的通道顺序为RGB
        然后opencv内部的顺序为BGR，因此这里需要调换顺序
        '''
        path=self.paths[index]
        img=Image.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        img=self.transforms(img)
        return img,self.labels[index]

    def __len__(self):
        return self.paths.shape[0]

# 实例化训练集
train_dataset=Mydataset(train_paths,train_labels,train_transforms)
# 封装
train_dataloader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True,drop_last=True)
# 实例化验证集
eval_dataset=Mydataset(eval_paths,eval_labels,eval_transforms)
# 封装
eval_dataloader=DataLoader(eval_dataset,batch_size=batch_size,shuffle=True,drop_last=True)
# 观察图片读取是否正确
img,label=next(train_dataloader())
print(img.shape)
show_imgs([img[3], img[4], img[5]])

### 定义网络

In [ ]:
class Model(nn.Layer):
    def __init__(self):
        super(Model, self).__init__()
        # 初始化参数
        nn.initializer.set_global_initializer(nn.initializer.XavierNormal(),nn.initializer.Constant(0.))
        self.model=nn.Sequential(
            # 调用预训练模型
            paddle.vision.models.resnet50(pretrained=True),
            nn.Linear(1000,128),
            nn.Sigmoid(),
            nn.Dropout(0.3),
            nn.Linear(128,12),
            nn.Dropout(0.3)
        )
    def forward(self,x):
        x=self.model(x)
        return x

x=paddle.randn([batch_size,3,224,224])
model=Model()
y=model(x)
print(y.shape)

### 开启训练

In [9]:
'''
开局使用AdamW加速收敛
当准确率达到一定程度后切换Momentum调优
'''
# optimizer=paddle.optimizer.AdamW(learning_rate=learning_rate,parameters=model.parameters())
scheduler=paddle.optimizer.lr.PolynomialDecay(learning_rate=learning_rate,decay_steps=20,end_lr=learning_rate/30)
optimizer=paddle.optimizer.Momentum(learning_rate=scheduler,parameters=model.parameters(),weight_decay=1e-2)

# 交叉熵损失
softmax_loss=paddle.nn.CrossEntropyLoss()
accuracy=paddle.metric.Accuracy()
# 最优准确率
max_score=0.
# 最优准确率对应轮数
ex_epoch=0
for epoch in range(num_epoch):
    model.train()
    for i,(data,label) in enumerate(train_dataloader()):
        summary=[]
        # 前向传播两次
        label_hat_A=model(data)
        label_hat_B=model(data)
        # cross entropy loss
        CE_loss=softmax_loss(label_hat_A,label)+softmax_loss(label_hat_B,label)
        # KL divergence loss
        KL_loss=0.5*(F.kl_div(F.softmax(label_hat_A,axis=-1),F.softmax(label_hat_B,axis=-1))+ \
                    F.kl_div(F.softmax(label_hat_B,axis=-1),F.softmax(label_hat_A,axis=-1)))
        # 损失加权求和
        loss=CE_loss+alpha*KL_loss
        # 反向传播
        loss.backward()
        # 更新参数
        optimizer.step()
        # 清除梯度
        optimizer.clear_gradients()
        if i%30==0:
            print("epoch:%d,i:%d,loss:%f"%(epoch,i,loss))
    
    model.eval()
    for j,(eval_data,eval_label) in enumerate(eval_dataloader()):
        summary=[]
        eval_label_hat=model(eval_data)
        eval_indexs=eval_label_hat.argmax(-1)
        eval_loss=softmax_loss(eval_label_hat,eval_label)
        correct=accuracy.compute(eval_label_hat,eval_label)
        accuracy.update(correct)
        acc=accuracy.accumulate()
        summary.append(acc)
        accuracy.reset()
    if sum(summary)/len(summary)>=max_score:
        max_score=sum(summary)/len(summary)
        ex_epoch=epoch
        paddle.save(model.state_dict(),"./cat.pdparams")
        print("[eval]saved params")
    print("eval: epoch:%d,loss:%f,acc:%f"%(epoch,eval_loss,sum(summary)/len(summary)))
    print("ex_epoch:%d,best acc:%f"%(ex_epoch,max_score))

### 加载测试集数据

In [10]:
test_imgs=[]
test_names=[]
for path in os.listdir(dir_path):
    name=dir_path+"/"+path
    img=Image.imread(name)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    if not(img is None):
        img=train_transforms(img)
        test_imgs.append(img)
        test_names.append(path)
test_imgs=paddle.to_tensor(test_imgs,dtype="float32")
print(test_imgs.shape)
print(len(test_names))

### 预测

In [11]:
state_dict=paddle.load("./cat.pdparams")
model=Model()
model.load_dict(state_dict)
model.eval()

final_result=[]
for i,data in enumerate(test_imgs):
    data=paddle.to_tensor(data)
    data=paddle.unsqueeze(data,axis=0)
    result=model(data)
    result=F.log_softmax(result)
    index=paddle.argmax(result,axis=-1)
    final_result.append(test_names[i]+","+str(index.numpy().item())+"\n")
with open("./result.csv","w") as f:
    f.writelines(final_result)

## 结果展示
**笔者大概训练了十分钟左右，可以看到效果还是很不戳滴**
![](https://ai-studio-static-online.cdn.bcebos.com/f4cdd28820034da08cf399317ffd403e769092331f6a485ebe01c113bf167fda)

## 总结
本项目主要讲了：
1. R-Drop的原理
2. R-Drop的代码实现
3. AI学习地图的实例代码，无脑运行即可通关，还不来玩^_^

## 关于作者
[我在AI Studio上获得钻石等级，点亮8个徽章，来互关呀~ ](https://aistudio.baidu.com/aistudio/personalcenter/thirdview/345331)

观众老爷们喜欢就给个三连吧，给俺一点小小的支持